## Code accompanyment to"Machine learning applied to a modern-Pleistocene petrographic dataset: The global prediction of sand modal composition (GloPrSM) model"
### J. Isaac Johnson, Glenn R. Sharman, Eugene Szymanski, and Xiao Huang
### University of Arkansas, Department of Geosciences
### Please direct questions and correspondance to gsharman@uark.edu

## Step 2: Load previously saved random forest models and generate predictions

### Import required modules

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import geopandas as gpd
from scipy.stats import skew
import time
from math import exp
import pickle
import glob
import os
import pathlib

### Function to compute inverse log-ratio transformation

In [ ]:
def invLogRatio(data_t):
    '''
    Calculates the inverse log-ratio transformation of an array of any shape (Vermeesch, 2019, section 3, Eqn. 2)
    
    Parameters
    ----------
    data_t : array with dimensions (m, n) containing log-ratio data
    
    Returns
    -------
    data_t_inv : inverse transformed array with dimensions (m, n+1)
    
    Notes
    -----
    Written by Isaac Johnson and Dr. Glenn Sharman (University of Arkansas)
    '''
    
    # Compute inverse logratio to get back to compositional (sum-to-one) data
    data_t_inv = np.zeros(shape=(data_t.shape[0],data_t.shape[1]+1))
    denominator = 1.
    
    for i in range(data_t.shape[1]):
        denominator += np.exp(data_t[:,i])
    for i in range(data_t.shape[1]):
        data_t_inv[:,i] = np.exp(data_t[:,i])/denominator
    data_t_inv[:,-1] = 1/denominator
    
    return data_t_inv

### Provide all inputs needed to run the code
After this, the code should be able to run through

In [ ]:
# Specify output folder used in Step 1 (current directory used by default)
base_path = os.getcwd() + '/v1.0'

# Base directory of where to save results (recommended same as base_path)
output_dir = os.getcwd() + '/v1.0'

# CSV of independent variables in upstream mapped drainages
wtrshd_ind_var = 'Watershed_output_lv8_ML_input.csv'

# Feature list exported when saving the models in Step 1
feature_list_dir = base_path + '/' + 'feature_list.csv'

# The labels of the log ratios you would like to predict
labels = ['FQ_QFL_IJ', 'LQ_QFL_IJ', 'QmQch_QmQpQch_IJ', 'QpQch_QmQpQch_IJ', 'FkFp_FpFk_IJ', 'LsLv_LvLsLm_IJ', 'LmLv_LvLsLm_IJ']

# The dictionary for how labels relate to QFL and the 8 QFL subcompositions (Qm, Qp, Qch, Fk, Fp, Ls, Lv, Lm)
systems_dict = {'QFL_IJ' : ['FQ_QFL_IJ', 'LQ_QFL_IJ'],
                'QmQpQch' : ['QmQch_QmQpQch_IJ', 'QpQch_QmQpQch_IJ'],
                'FpFk' : ['FkFp_FpFk_IJ'],
                'LvLsLm' : ['LmLv_LvLsLm_IJ', 'LsLv_LvLsLm_IJ']
               }

final_columns = ['F','L','Q','Qm','Qp','Qch','Fk','Fp','Lm','Ls','Lv']
methods = ['avg','med','std','skew','min','max','p025','p25','p75','p975']

### Load necessary files

In [ ]:
# Recursively creates the directory and does not raise an exception if the directory already exists
pathlib.Path(output_dir).mkdir(parents=True, exist_ok=True)

feature_list = list(pd.read_csv(feature_list_dir)['Inputs'])
print('These are the inputs:',feature_list)

wtrshd_data = pd.read_csv(wtrshd_ind_var)

input_data = wtrshd_data[feature_list]
print('There are',len(input_data.columns),'independent variables in the model')
print('There are',len(input_data),'watersheds to be predicted')

# Export the results
input_data.to_csv(output_dir +'/' + 'input_data.csv') # Save the input data to CSV

### Load models and generate predictions
Note: This takes 2 to 8+ hours, depending on the number of watersheds you are predicting and the number of models generated in Step 1

In [ ]:
for label in labels:
    models_to_import = []
    os.chdir(base_path+'/'+'models'+'/'+label)

    var_df = pd.DataFrame()

    # Get the model file paths
    for root, dirs, files in os.walk(base_path+'/'+'models'+'/'+label):
        for file in glob.glob("*.sav"):
            models_to_import.append(os.path.join(root,file))
            
    # Load the models
    for i in range(len(models_to_import)):
        start = time.time()

        model = pickle.load(open(models_to_import[i], 'rb'))
        pred = model.predict(input_data)

        var_df.loc[:,'{}_globe_{}'.format(label, i)] = pred
        print('Finished:',models_to_import[i], 'Time: {} sec'.format(round(time.time()-start,1)))

    # Recursively creates the directory and does not raise an exception if the directory already exists
    pathlib.Path(output_dir+'/'+'variance_raw').mkdir(parents=True, exist_ok=True)
        
    var_df.to_csv(output_dir+'/'+'variance_raw'+'/{}_variance_raw_rlf.csv'.format(label),index=False)

### Load predictions

In [ ]:
os.chdir(output_dir+'/'+'variance_raw')
files_to_import = []

for root, dirs, files in os.walk(output_dir+'/'+'variance_raw'):
    for file in glob.glob("*.csv"):
        files_to_import.append(os.path.join(root,file))

print('>>> files to import:', len(files_to_import))
files_to_import[:]

### Calculate log statistics

In [ ]:
start = time.time()

for file in files_to_import[:]:
    
    var_df = pd.DataFrame()
    var_data = pd.read_csv(file)
    ratio = file.split('/')[-1].split('_v')[0] # For Mac
    #ratio = file.split('\\')[-1].split('_v')[0] # For PC
    print('Currently calculating',ratio)
    
    var_df['HYBAS_ID'] = wtrshd_data['Wtrshd_ID']
    var_df.loc[:,'{}_avg'.format(ratio)] = np.asarray(np.mean(var_data,axis=1))
    var_df.loc[:,'{}_med'.format(ratio)] = np.median(var_data,axis=1)
    var_df.loc[:,'{}_std'.format(ratio)] = np.asarray(np.std(var_data,axis=1))
    var_df.loc[:,'{}_skew'.format(ratio)] = skew(var_data,axis=1)

    runtime = np.zeros(len(wtrshd_data))
    var_min = []
    var_max = []
    var_p025 = []
    var_p250 = []
    var_p750 = []
    var_p975 = []
    
    for i in range(len(wtrshd_data)):
        loop_start = time.time()
        
        var_min.append(var_data.loc[i].min())
        var_max.append(var_data.loc[i].max())
        var_p025.append(np.percentile(var_data.loc[i], q=2.5))
        var_p250.append(np.percentile(var_data.loc[i], q=25))
        var_p750.append(np.percentile(var_data.loc[i], q=75))
        var_p975.append(np.percentile(var_data.loc[i], q=97.5))
        
        runtime[i]=time.time()-loop_start
        if (i+1)%10000==0:
            print(round(i/len(wtrshd_data)*100,4),'% completed.',round(np.mean(runtime[:i]),6),'seconds per basin')

    var_df.loc[:,'{}_min'.format(ratio)] = var_min
    var_df.loc[:,'{}_max'.format(ratio)] = var_max
    var_df.loc[:,'{}_p025'.format(ratio)] = var_p025
    var_df.loc[:,'{}_p25'.format(ratio)] = var_p250
    var_df.loc[:,'{}_p75'.format(ratio)] = var_p750
    var_df.loc[:,'{}_p975'.format(ratio)] = var_p975
    
    # Recursively creates the directory and does not raise an exception if the directory already exists
    pathlib.Path(output_dir+'/'+'log_ratio_stats').mkdir(parents=True, exist_ok=True)
    
    var_df.to_csv(output_dir+'/'+'log_ratio_stats'+'/{}_stats.csv'.format(ratio),index=False)
print(time.time()-start)

### Group log ratio stats by individual statistics

In [ ]:
os.chdir(output_dir+'/'+'log_ratio_stats')
files_to_import = []

for root, dirs, files in os.walk(output_dir+'/'+'log_ratio_stats'):
    for file in glob.glob("*.csv"):
        files_to_import.append(os.path.join(root,file))

print('>>> files to import:', len(files_to_import))
files_to_import[:]

In [ ]:
# Make a list of files to import
ratio_csvs = []
for file in files_to_import:
    ratio_csvs.append(pd.read_csv(file))

In [ ]:
stats = ['avg','med','std','skew','min','max','p025','p25','p75','p975']
ratios = [file.split('/')[-1].split('stats/')[0] for file in files_to_import]  # Mac
#ratios = [file.split('\\')[-1].split('stats\\')[0] for file in files_to_import]  # PC
ratios = [x[:(len(x)-10)] for x in ratios]

var_data_dict = {'LR_data' : ratio_csvs}

for key in var_data_dict:
    for stat in stats:
        print('Starting',stat)

        method_df = pd.DataFrame()
        method_df['HYBAS_ID'] = var_data_dict[key][0]['HYBAS_ID']

        for i in range(len(ratios)):
            LR = var_data_dict[key][i].loc[:,'{}_{}'.format(ratios[i], stat)]
            method_df['{}_{}'.format(ratios[i],stat)] = LR
        
        print(stat, np.array(method_df).shape)
        
        # Recursively creates the directory and does not raise an exception if the directory already exists
        pathlib.Path(output_dir+'/'+'log_ratio_by_stat_type').mkdir(parents=True, exist_ok=True)        
        
        method_df.to_csv(output_dir+'/'+'log_ratio_by_stat_type'+'/GloPrSM_LR_{}.csv'.format(stat),index=False)

### Inverse transform to ternary values

In [ ]:
os.chdir(output_dir+'/'+'log_ratio_by_stat_type')
files_to_import = []

for root, dirs, files in os.walk(output_dir+'/'+'log_ratio_by_stat_type'):
    for file in glob.glob("*.csv"):
        files_to_import.append(os.path.join(root,file))

print('>>> files to import:', len(files_to_import))
files_to_import[:]

In [ ]:
QFL = np.zeros(shape=(len(wtrshd_data),2))
QmQpQch = np.zeros(shape=(len(wtrshd_data),2))
FkFp = np.zeros(shape=(len(wtrshd_data),1))
LmLsLv = np.zeros(shape=(len(wtrshd_data),2))

all_ratios = [QFL, QmQpQch, FkFp, LmLsLv]
oct_ratios = [QmQpQch, FkFp, LmLsLv]
oct_tax = np.zeros(shape=(len(wtrshd_data),11))
oct_tax_norm = np.zeros(shape=(len(wtrshd_data),11))

# Recursively creates the directory and does not raise an exception if the directory already exists
pathlib.Path(output_dir+'/'+'IJ_QFL_relative_vals').mkdir(parents=True, exist_ok=True)    
pathlib.Path(output_dir+'/'+'IJ_QFL_normalized_vals').mkdir(parents=True, exist_ok=True)    

for i, file in enumerate(files_to_import[:]):
    data = pd.read_csv(file)
    method = file.split('_')[-1].split('.')[0]
    print(method)
    
    index1, index2 = 0, 0
    for j, key in enumerate(systems_dict): # iterate through each system
        for k, item in enumerate(systems_dict[key]): # iterate through each system's log ratios
            all_ratios[j][:,k] = data[item+'_'+method]
        
        inv_LR = invLogRatio(all_ratios[j])
        index2+=inv_LR.shape[-1]
        oct_tax[:,index1:index2] = inv_LR
        
        for index in range(index1,index2):
            if index1 == 0:
                oct_tax_norm[:,index] = inv_LR[:,index]
            elif index1 == 3: # quartz
                oct_tax_norm[:,index] = oct_tax[:,index]*oct_tax[:,2]
            elif index1 == 6: # feldspar
                oct_tax_norm[:,index] = oct_tax[:,index]*oct_tax[:,0]
            elif index1 == 8: # lithics
                oct_tax_norm[:,index] = oct_tax[:,index]*oct_tax[:,1]
        
        print(index1, index2)
        index1+=inv_LR.shape[-1]
    
    var_df = pd.DataFrame(oct_tax, columns=[col+'_'+method for col in final_columns])
    var_df_norm = pd.DataFrame(oct_tax_norm, columns=[col+'_'+method for col in final_columns])
    var_df['HYBAS_ID'], var_df_norm['HYBAS_ID'] =  data['HYBAS_ID'], data['HYBAS_ID']
    
    var_df.to_csv(output_dir+'/'+'IJ_QFL_relative_vals'+'/octonary_var_{}.csv'.format(method),index=False)
    var_df_norm.to_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_{}.csv'.format(method),index=False)
    print()

In [ ]:
p025 = pd.read_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_p025.csv')
p25 = pd.read_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_p25.csv')
p75 = pd.read_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_p75.csv')
p975 = pd.read_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_p975.csv')
inner_50 = pd.DataFrame()
inner_95 = pd.DataFrame()

for i in range(len(p025.columns[:-1])):
    print(final_columns[i])
    inner_50.loc[:,final_columns[i]+'_i50'] = abs(p75.iloc[:,i]-p25.iloc[:,i])
    inner_95.loc[:,final_columns[i]+'_i95'] = abs(p975.iloc[:,i]-p025.iloc[:,i])

inner_50['HYBAS_ID'] = data['HYBAS_ID']
inner_95['HYBAS_ID'] = data['HYBAS_ID']
inner_50.to_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_i50.csv',index=False)
inner_95.to_csv(output_dir+'/'+'IJ_QFL_normalized_vals'+'/octonary_var_i95.csv',index=False)

### Export results as a CSV

In [ ]:
os.chdir(output_dir+'/'+'IJ_QFL_normalized_vals')
files_to_import = []

for root, dirs, files in os.walk(output_dir+'/'+'IJ_QFL_normalized_vals'):
    for file in glob.glob("*.csv"):
        files_to_import.append(os.path.join(root,file))

print('>>> files to import:', len(files_to_import))
files_to_import[:]

In [ ]:
df_to_export = pd.DataFrame()
for file in files_to_import:
    new_data = pd.read_csv(file)
    for col in new_data.columns:
        df_to_export[col] = new_data[col]

In [ ]:
wtrshd_data = wtrshd_data.rename(columns={"Wtrshd_ID": "HYBAS_ID"})

In [ ]:
# Only exporting the median and inner 95th percentile
columns_to_export = ['HYBAS_ID','F_med','L_med','Q_med','Qm_med','Qp_med','Qch_med','Fk_med','Fp_med','Lm_med','Ls_med','Lv_med',
                    'F_i95','L_i95','Q_i95','Qm_i95','Qp_i95','Qch_i95','Fk_i95','Fp_i95','Lm_i95','Ls_i95','Lv_i95']

In [ ]:
wtrshd_merge = wtrshd_data[['HYBAS_ID','Area_sq_km']].merge(df_to_export[columns_to_export],on='HYBAS_ID')

In [ ]:
wtrshd_merge.to_csv(output_dir+'/'+'wtrshd_data_wQFL.csv')

### Join results with the BasinATLAS level 08 shapefile and export as shapefile
Note: you need to download the BasinATLAS v10 level-08 shapefile (~875 MB) from www.hydrosheds.org for the code below to work

In [ ]:
# Need to download the BasinATLAS (v10) level 8 shapefile and specify filepath below
wtrshd_shp = gpd.read_file('BasinATLAS_v10_lev08.shp')

In [ ]:
wtrshd_shp_wQFL = wtrshd_shp[['HYBAS_ID','geometry']].merge(wtrshd_merge, on='HYBAS_ID')

In [ ]:
wtrshd_shp_wQFL.to_file(output_dir+'/'+'wtrshd_shp_wQFL.shp')